# Imports & Options

In [1]:
%load_ext autoreload
%autoreload 2

import crispr as cr 
from crispr.crispr_class import Crispr
import celltypist
import pertpy as pt
import pandas as pd
import numpy as np
from config import (
    files_data, assays_data, col_cell_type_data, col_gene_symbols_data,
    col_perturbed_data, col_condition_data, key_control_data, 
    key_treatment_data, col_split_by_data, 
    col_batch_data, col_sample_id_data, 
    kws_process_guide_rna_data, col_guide_rna_data, col_num_umis_data)

# Options
pd.options.display.max_columns = 100
skip_preprocessing_clustering = False  # to skip these steps
print(dir(cr.ax))
file = "CRISPRi_scr"
# file = "CRISPRi_ess"
# file = "ECCITE"
# file = "augur_ex"
# file = "coda"
# file = "screen"
# file = "perturb-seq"
kws_pca = dict(n_comps=None, use_highly_variable=True)
kws_neighbors = None  # passed to neighbors function
kws_umap = dict(min_dist=0.3)  # passed to UMAP fx
kws_cluster = dict(resolution=0.5)  # passed to louvain/leiden fx

#  Set Arguments
if file in assays_data:
    if isinstance(assays_data[file], str) or assays_data[file] is None:
        assays_data[file] = [assays_data[file], None]
    assay, assay_protein = assays_data[file]
else:
    assay, assay_protein = None
file_path = files_data[file]
col_split_by = col_split_by_data[file]
kwargs_init = dict(assay=assay, assay_protein=assay_protein,
                   col_gene_symbols=col_gene_symbols_data[file],  
                   col_cell_type=col_cell_type_data[file], 
                   col_sample_id=col_sample_id_data[file], 
                   col_batch=col_batch_data[file],
                   col_perturbed=col_perturbed_data[file], 
                   col_guide_rna=col_guide_rna_data[file], 
                   col_num_umis=col_num_umis_data[file],
                   kws_process_guide_rna=kws_process_guide_rna_data[file],
                   col_condition=col_condition_data[file], 
                   key_control=key_control_data[file], 
                   key_treatment=key_treatment_data[file],
                   remove_multi_transfected=True)
print(files_data)
if file == "default_fast":
    skip_preprocessing_clustering = True
print("\n\n\nInitialization keyword arguments:\n", kwargs_init)

['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'analyze_composition', 'cluster', 'clustering', 'compute_distance', 'find_markers', 'perform_augur', 'perform_celltypist', 'perform_differential_prioritization', 'perform_gsea', 'perform_mixscape', 'perturbations']
{'CRISPRi_scr': {'directory': '/home/asline01/projects/crispr/examples/data/crispr-screening/HH03'}, 'CRISPRi_wgs': '/home/asline01/projects/crispr/examples/data/replogle_2022_k562_gwps.h5ad', 'CRISPRi_ess': '/home/asline01/projects/crispr/examples/data/replogle_2022_k562_esss.h5ad', 'pool': '/home/asline01/projects/crispr/examples/data/norman_2019_raw.h5ad', 'bulk': '/home/asline01/projects/crispr/examples/data/burczynski_crohn.h5ad', 'screen': '/home/asline01/projects/crispr/examples/data/dixit_2016_raw.h5ad', 'perturb-seq': '/home/asline01/projects/crispr/examples/data/adamson_2016_upr_perturb_seq.h5ad', 'ECCITE': '/home/asline01/projects/crispr/examples/

# Setup

Welcome to Crispr!

<u> Argument Conventions: </u>

Certain arguments used throughout the `crispr` package (including outside the `crispr.crispr_class.Crispr()` class), hold to conventions intended to foster readability, maintain consistency, and promote clarity, both for end-users and future developers.

* Arguments starting in `col_` and `key_`
    - The "col_" prefix indicates that an argument refers to a column name (often in `.adata.obs` or `.adata.var`), while the "key_" prefix means you're meant to specify a type of entry in a column. For instance, assume the column "condition" contains the names of different experimental conditions (drug A, drug B, drug C). In a function where you want to compare, for instance, drug A vs. control, you would specify `key_treatment="drug A"` and k
    - These names may 
        * already exist (or will exist in the `.adata` attribute immediately upon creating the AnnData object from the data file) or 
        * may yet to be created, namely, after object initialization by running the object's methods. Thus, you may specify what you want certain columns to be named (e.g., the binary perturbed/non-perturbed column) or what entries within a column will be called (e.g., "Control" for rows within the `col_control` corresponding to cells that have control guide RNA IDs in `col_guide_rna`), for aesthetics, customizability to your design/interpretability, and/or to avoid duplicating pre-existing names.
    - These arguments will be entered as items (with the argument names as keys) in dictionaries stored in the object attributes `._columns` and `._keys`, respectively.
    - These arguments will often be passed by default (or will force them as specifications) to various object methods.
    - In certain methods, you can specify a new column to use just for that function. For instance, if you have a column containing CellTypist annotations and want to use those clusters instead of the "leiden" ones for the `run_dialogue()` method, you can specify in that method (`run_dialogue(col_cell_type="majority_voting")`) without changing the attribute (`self._columns`) that contains your original specification here.

<!-- break -->

* `col_perturbed` (binary) vs. `col_condition` (can have >= 3 categories)
    - In the `Crispr` class object, `col_perturbed` is meant to be a binary column that has `key_control` as the entry for control rows and `key_treatment` for all other experimental conditions. 
        * For instance, for a CRISPR design targeting more than one gene, `col_perturbed` would contain only `key_treatment` (i.e., all perturbed cells, regardless of targeted gene) while `col_condition` would contain entries specifying the particular gene(s) targeted (or `key_control`).
        * A drug design targeting more than one gene, `col_perturbed` would contain only `key_treatment` (i.e., all perturbed cells, regardless of targeted gene) while `col_condition` would contain entries specifying the particular gene(s) targeted (or `key_control`).
        * If the design only targets one gene/has one treatment conditions/etc., these columns would simply be equivalent.
    - In the `Crispr` class object, it is created during object initialization as a column (named after your specification of `col_perturbed`) in `.obs`. All rows in `.obs[col_condition]` that do not = `key_control` will be set as `key_treatment`.
    - In the `crispr` package more broadly, if a function calls for a `col_perturbed` argument, that indicates that it works with binary categories only. If it is fed a column with three or more categories, it will either subset the data to include only rows where that column = `key_treatment` or `key_control` (desirable behavior if you want to compare only a subset of the existing conditions, but undesirable if you want to look at, say, any drug vs. control, where the desired "drug" category consists of rows where the column = "drug A" and "drug B"), or it will throw an error.

<!-- break -->

* `col_guide_rna`

### Initialization Method Arguments

* `file_path` **(str, AnnData, or dictionary)**: Path or object containing data. Used in initialization to create the initial `self.adata` attribute (an AnnData or MuData object). Either
    - a path to a 10x directory (with matrix.mtx.gz, barcodes.tsv.gz, features.tsv.gz),
    - a path to an .h5ad or .mu file (Scanpy/AnnData/Muon-compatible), 
    - an AnnData or MuData object (e.g., already loaded with Scanpy or Muon), or
    - a dictionary containing keyword arguments to pass to  `crispr.pp.combine_matrix_protospacer()` (in order to load information about perturbations from other file(s); see function documentation).

<!-- break -->

* `assay` **(str, optional)**: Name of the gene expression assay if loading a multi-modal data object (e.g., "rna"). Defaults to None (i.e., `self.adata` is single-modal).

* `assay_protein` **(str, optional)**: Name of the assay containing the protein expression modality, if available. For instance, if "adt", `self.adata["adt"]` would be expected to contain the AnnData object for the protein modality. ONLY FOR MULTI-MODAL DATA for certain bonus visualization methods. Defaults to None.

* `col_gene_symbols` **(str, optional)**: Column name in `.var` for gene symbols. Defaults to "gene_symbols".

* `col_cell_type` **(str, optional)**: Column name in `.obs` for cell type. Defaults to "leiden" (anticipating that you will run `self.cluster(...)` with `method_cluster="leiden"`). This column may be
    - pre-existing in data (e.g., pre-run clustering column or manual annotations), or 
    - expected to be created via `Crispr.cluster()`. 

<!-- break -->

* `col_sample_id` **(str, optional)**: Column in `.obs` with sample IDs. Defaults to "standard_sample_id".

* `col_batch` **(str, optional)**: Column in `.obs` with batch IDs. Defaults to None.

* `col_condition` **(str, optional)**: Either the name of an existing column in `.obs` indicating the experimental condition to which each cell belongs or (for CRISPR designs) the **desired** name of the column that will be created from `col_guide_rna` to indicate the gene(s) targeted in each cell.
    - If there are multiple conditions besides control (e.g., multiple types of drugs and/or exposure times, multiple target genes in CRISPR), this column distinguishes the different conditions, in contrast to `col_perturbed` (a binary treated/perturbed vs. control indicator). 
    - In CRISPR designs (i.e., when `col_guide_rna` is specified), this column will be where each guide RNA's target gene will be stored, whether pre-existing (copied directly from `col_guide_rna` if `kws_process_guide_rna` is None) or created during the Crispr object initialization by passing `col_guide_rna` and `kws_process_guide_rna` to `crispr.pp.filter_by_guide_counts()` in order to convert particular guide RNA IDs to their target(s) (e.g., STAT1-1|CCL8-2-1|NegCtrl32a => STAT1|CCL8|Control). Defaults to None.
    - For non-CRISPR designs (e.g., pharmacological treatment):
        - This column should exist in the AnnData or MuData object (either already available upon simply reading the specified file with no other alterations, or as originally passed to the initialization method if given instead of a file path).
        - It should contain a single `key_control`, but it can have multiple categories of other entries that all translate to `key_treatment` in `col_perturbed`.
        - If you have multiple control conditions, you should pass an already-created AnnData object to the `file_path` argument of the `Crispr` class initialization method after adding a separate column with a name different from those specified in any of the other column arguments. You can then pass that column name manually to certain functions' `col_control` arguments and specify the particular control condition in `key_control`.
    - In most methods, `key_control` and `key_treatment`, as well as `col_perturbed` or `col_condition` (for methods that don't require binary labeling), can be newly-specified so that you can compare different conditions within this column. If the argument is named `col_perturbed`, passing a column with more than two categories usually results in subsetting the data to compare only the two conditions specified in `key_treatment` and `key_control`. The exception is where there is a `key_treatment_list` or similarly-named argument.

<!-- break -->

* `col_perturbed` **(str, optional)**: Column in `.obs` where class methods will be able to find the binary experimental condition variable. It will be created during `Crispr` object initialization as a binary version of `col_condition`. Defaults to "perturbation". For CRISPR designs, all entries containing the patterns specified in `kws_process_guide_rna["key_control_patterns"]` will be changed to `key_control`, and all cells with targeting guides will be changed to `key_treatment`.

<!-- break -->

* `col_guide_rna` **(str, optional)**: Column in `.obs` with guide RNA IDs. Defaults to "guide_ids". This column should always be specified for CRISPR designs and should NOT be specified for other designs. 
    - If only one kind of guide RNA is used, then this should be a column containing the name of the gene targeted (for perturbed cells) and the names of any controls, and `key_treatment` should be the name of the gene targeted. Then, `col_condition` will be a direct copy of this column.
    - Entries in this column should be either gene names in `self.adata.var_names` (or `key_control` or one of the patterns in `kws_process_guide_rna["key_control_patterns"]`), plus, optionally, suffixes separating guide #s (e.g., STAT1-1-2, CTRL-1) and/or with a character that splits separate guide RNAs within that cell (if multiply-transfected cells are present). These characters should be specified in `kws_process_guide_rna["guide_split"]` and `kws_process_guide_rna["feature_split"]`, respectively. For instance, they would be "-" and "|", if `col_guide_rna` entries for a cell multiply transfected by two sgRNAs targeting STAT1, two control guide RNAs, and a guide targeting CCL5  would look like "STAT1-1-1|STAT1-1-2|CNTRL-1-1|CCL5-1".
    - Currently, everything after the first dash (or whatever split character is specified) is discarded when creating `col_target_genes`, so keep that in mind.
    - This column will be stored as `<col_guide_rna>_original` if `kws_process_guide_rna` is not None, as that will result in a processed version of this column being stored under `self.adata.obs[<col_guide_rna>]`.

<!-- break -->

* `col_num_umis` **(str, optional)**: Name of column in `.obs` with the UMI counts. This should be specified if `kws_process_guide_rna` is not None. For designs with multiply-transfected cells, it should follow the same convention established in `kws_process_guide_rna["feature_split"]`. Defaults to "num_umis".

* `key_control` **(str, optional)**: The label that is or will be in `col_condition`, `col_guide_rna`, and `col_perturbed` indicating control rows. Defaults to "NT". Either 
    - exists as entries in pre-existing column(s), or
    - is the name you want the control entries (detected using `.obs[<col_guide_rna>]` and `kws_process_guide_rna["key_control_patterns"]`) to be categorized as control rows under the new version(s) of `.obs[<col_guide_rna>]`, `.obs[<col_target_genes>]`, and/or `.obs[<col_perturbed>]`. For instance, entries like "CNTRL-1", "NEGCNTRL", "Control", etc. in `col_guide_rna` would all be keyed as "Control" in (the new versions of) `col_target_genes`, `col_guide_rna`, and `col_perturbed` if you specify `key_control="Control` and `kws_process_guide_rna=dict(key_control_patterns=["CTRL", "Control"])`. 

<!-- break -->

* `key_treatment` **(str, optional)**: What entries in `col_perturbed` indicate a treatment condition (e.g., drug administration, CRISPR knock-out/down) as opposed to a control condition? This name will also be used for Mixscape classification labeling. Defaults to "KO".

<!-- break -->

* `key_nonperturbed` **(str, optional)**: What will be stored in the `mixscape_class_global` and related columns/labels after running Mixscape methods. Indicates cells without a detectible perturbation. Defaults to "NP".

* `kws_process_guide_rna` (dict, optional): Dictionary of keyword arguments to pass to `crispr.pp.filter_by_guide_counts()`. (See below and crispr.processing.preprocessing documentation). Defaults to None (no processing will take place, in which case BE SURE THAT `col_target_genes` already exists in the data once loaded and contains the already-filtered, summed up, generic gene-named, etc. versions of the guide RNA column). Keys of this dictionary should be:
    - key_control_patterns (list, optional): List (or single string) of patterns in guide RNA column entries that correspond to a control. For instance, if control entries in the original `col_guide_rna` column include `NEGCNTRL` and `Control.D`, you should specify ['Control', 'CNTRL'] (assuming no non-control sgRNA names contain those patterns). If blank entries should be interpreted as control guides, then include np.nan/numpy.nan in this list. Defaults to None, which turns to [np.nan].
    - `max_percent_umis_control_drop` (int, optional): If control UMI counts are $<=$ this percentage of the total counts for that cell, and if a non-control sgRNA is also present and meets other filtering criteria, then consider that cell pseudo-single-transfected (non-control gene). Defaults to 75.
    - `min_percent_umis` (int, optional): sgRNAs with counts below this percentage will be considered noise for that guide. Defaults to 40.
    - `feature_split` (str, optional): For designs with multiple guides, the character that splits guide names in `col_guide_rna`. For instance, "|" for `STAT1-1|CNTRL-1|CDKN1A`. Defaults to "|". If only single guides, you should set to None.
    - `guide_split` (str, optional): The character that separates guide (rather than gene target)-specific IDs within gene. For instance, guides targeting STAT1 may include STAT1-1, STAT1-2-1, etc.; the argument would be "-" so the function can identify all of those as targeting STAT1. Defaults to "-".

<!-- break -->

* `remove_multi_transfected` (bool, optional): In designs with multiple guides per cell, remove multiply-transfected cells (i.e., cells where more than one target guide survived 
    application of any filtering criteria set in `kws_process_guide_rna`). If `kws_process_guide_rna["max_percent_umis_control_drop"]` is greater than 0, then cells with one target guide and control guides which together make up less than `max_percent_umis_control_drop`% of total UMI counts will be considered pseudo-single-transfected for the target guide. Defaults to True. Some functionality may be limited and/or problems occur if set to False and if multiply-transfected cells remain in data. 

<!-- break -->

### Crispr Object Properties

#### Major Attributes

#### Major Attribute Descriptions

* `.adata`: AnnData object. Columns or other objects created in the course of running certain methods may also be stored in its various attributes. Below are listed some of the major attributes of `.adata`. Note that for multi-modal data (self._assay is not None), some of these attributes may need to be accessed by `.adata[self._assay].<attribute>`, but for brevity, we'll refer to `.adata` here. Not all will/have to be present, except `.X`, `.obs`, and `.var`.
    -  `.X`: Sparse matrix of data originally passed to the function to create an AnnData object (e.g., from CellRanger output).
    -  `.layers`: Contains different versions of `adata.X`, such as scaled data (`adata.layers["scaled"]`) or that created by calculating a 
                  Mixscape perturbation signature (`adata.layers["X_pert"]`, by default).
    -  `.obs`: pandas.DataFrame of observations (rows=cells). You can store additional data about individual cells/samples/etc. here by assigning a new column.
    -  `.obsm`: xxxxxxxxxxxxxxxxxxxxxxxxxx
    -  `.obsm`: xxxxxxxxxxxxxxxxxxxxxxxxxx
    -  `.var`: pandas.DataFrame of observations (rows=cells). You can store additional data about individual cells/samples/etc. here by assigning a new column. 
            Often contains the gene symbols and EnsemblIDs (either of which is often the index/`.var_names`),
            "feature_types" (e.g., "Gene Expression"), and, after preprocessing, may contain columns such as the number of cells expressing that feature ("n_cells"),
            whether that feature is a mitochonrial ("mt") and/or highly variable ("highly_variable") gene, mean and total counts, percent dropout, means, dispersions, 
            and normalized versions of these metrics.
    -  `.obs_names`: Row indices of `.obs` (e.g., cell barcodes). Changing this attribute changes this index, and has other potential benefits/consequences.
    -  `.var_names`: Row indices of `.var` (i.e., gene names). Changing this attribute changes this index, and has other potential benefits/consequences.
    -  `.n_obs`: Number of observations (i.e., cells).
    -  `.n_vars`: Number of features (i.e., genes/proteins/etc.).
*
*

#### Accessing AnnData and Attributes Directly and Using Aliases

The AnnData object is stored in the attribute `adata`, so if your object is called `self`, you can access it using `self.adata`. (For examples going forward, we will assume the object is called `self`, but you can substitute any name you want by assigning the Crispr object to some other name instead.)

If you have multiple modalities, you can access the gene expression modality using either `self.adata[self._assay]` (having specified assay=the name of the RNA modality in your AnnData, which is usually "rna," in the `Cripsr()` initialization method call when you first create your object) or using the alias `self.rna`.

Similarly, you can access the `.obs` attribute of your AnnData using `self.adata.obs` directly, or simply `self.obs`. The same holds true for `.var`, `.uns`, and `.obsm`, except that while `self.obs` for multi-modal objects holds the overall `.obs` attribute (with prefixes by assay on the column names, e.g., rna:num_umis, adt:num_umis), the others listed call the gene expression modality attribute directly (e.g., `self.adata[self._assay].uns`).

If you have multi-modal data in `self.adata`, it's convenient to access the `.obs` attribute specifically of your AnnData's gene expression modality using the alias `self.rna.obs` instead of the long-form `self.adata[self._assay].obs`.

These aliases are not only convenient for their brevity, but also allow for a more generalizable way to call specific objects. For instance, if you wanted to write a script that frequently calls the `.obs` attribute of the RNA data, and you want it to work for both uni- and multi-modal data, instead of repeatedly writing, for example:

```
if self._assay is None:
    custom_function(self.adata[self._assay].obs)
else:
    custom_function(self.adata[self._assay].obs)
```
you may simply say `self.obs.rna`, knowing it will work whether or not multiple assays exist in the object's AnnData attribute.

Finally, this approach saves memory: All these versions of the attribute are stored in a single place in memory so you can call the attributes in various ways without duplicating them and taking up more space.

#### Major Methods

Here are the methods (applicable to scRNA-seq generally, not just perturbations) **in order** of a typical workflow _(replace ... with argument specifications)_:

* `self.preprocess(...)`: Perform filtering, normalization, scaling, quality control analysis, selection for highly variable genes, regressing out confounds, and intial exploration of the data (e.g., cell counts, mitochondrial counts, etc.).
* `self.cluster(...)`: Perform dimensionality reduction (e.g., PCA) and clustering (Louvain or Leiden), as well as related visualization.
* `self.plot(...)`: Create additional basic plots (e.g., dot, matrix, and violin gene expression plots).

The following perturbation-specific methods can be executed optionally and in any order:

* `self.run_augur(...)`: Score and plot how strongly different cell types responded to perturbation(s). This score is operationalized as the accuracy with which a machine learning model can use gene expression data to predict the perturbation condition to which cells of a given type belong. Augur provides scores aggregated across cells of a given type rather than for individual cells.
* `self.run_mixscape(...)`: Quantify and plot the extent to which individual cells responded to CRISPR perturbation(s), and identify which perturbation condition cells were not detectibly perturbed in terms of their gene expression.
* `self.compute_distance(...)`: Calculate and visualize various distance metrics that quantify the similarity in gene expression profiles across perturbation conditions.
* `self.run_composition_analysis(...)`: Analyze and visualize shifts in the cell type composition across perturbation conditions.
* `self.run_dialogue(...)`: Create plots showing multi-cellular programs.

## Instantiate Object

This is more code than an end-user will actually need. `self = Crispr(file_path)` is the bare minimum needed (though you must specify some column and entry label names, such as the column name for the gene symbols, if they aren't the same as the defaults).

End users can simply pass to the `file` argument of `Crispr()` a 10x CellRanger output directory (or a dictionary of appropriate protospacer-related arguments if perturbation-related data aren't included in the .mtx file; see the `Crispr.preprocessing.combine_matrix_protospacer()` function for details) or .h5ad file if desired.

The extra code is just for generalizability across example datasets because in certain cases, we want to pass an AnnData Scanpy object because there are specific manipulations made to certain public datasets in the `config.load_example_data`` function meant to decrease computation time/memory usage, etc.

In [3]:
print(f"Initialization arguments (from config):\n\n{kwargs_init}")
if isinstance(file, str) and file in ["ECCITE", "Adamson"]:
    self = Crispr(
        pt.data.papalexi_2021() if file == "ECCITE" else
        pt.data.adamson_2016_upr_perturb_seq(), **kwargs_init)
else:
    self = Crispr(file_path, **kwargs_init)
# self.adata[self._assay].obs if self._assay else self.adata.obs  # long-form
self.obs  # easier way

Initialization arguments (from config):

{'assay': None, 'assay_protein': None, 'col_gene_symbols': 'gene_symbols', 'col_cell_type': 'leiden', 'col_sample_id': None, 'col_batch': None, 'col_perturbed': 'perturbation', 'col_guide_rna': 'feature_call', 'col_num_umis': 'num_umis', 'kws_process_guide_rna': {'feature_split': '|', 'guide_split': '-', 'key_control_patterns': ['CTRL']}, 'col_condition': 'target_gene_name', 'key_control': 'NT', 'key_treatment': 'KD', 'remove_multi_transfected': True}


<<<INITIALIZING CRISPR CLASS OBJECT>>>



# Preprocess and Cluster

See `self.figures[run_label]["clustering"]` (object attribute) or method output for figures.

You must set use_highly_variable to False if you don't want to filter by HVGs
and if you did not set kws_hvg to None or False in `.preprocess()`


In that scenario, the 'highly_variable' column in  will be created in `.obsm` 
during `.preprocess()` and then used by default in `.cluster`.

In [ ]:
if skip_preprocessing_clustering is False:  # if not already processed & clustered
    
    # Arguments
    print(f"UMAP Keywords: {kws_umap}")
    
    # Preprocess
    process_kws = dict(
        kws_hvg=dict(min_mean=0.0125, max_mean=10, min_disp=0.5),
        target_sum=1e4, max_genes_by_counts=6000, 
        cell_filter_pmt=[0, 5],  # range of mitchondrial gene % allowed
        cell_filter_ngene=[200, None],  # cells: min/max genes expressed
        cell_filter_ncounts=[3,6000],  # cells: min/max read range
        gene_filter_ncell=[3, None],  # genes: min/max cell count range
        scale=10, regress_out=None)
    self.preprocess(**process_kws, kws_umap=kws_umap)  # preprocessing
    
    # Cluster 
    self.cluster(method_cluster="leiden", kws_neighbors=kws_neighbors,
                 kws_pca=kws_pca, kws_umap=kws_umap,  kws_cluster=kws_cluster)
    
self.obs.rna

### CellTypist Cell Cluster Annotation

You can see what models are available with this code:
```
import celltypist
celltypist.models.models_description().set_index("model").description.apply(lambda x: print(x))
celltypist.models.models_description()
```

Or it will do it for you automatically if you feed it an unknown model:

```
self.annotate_clusters(model="kate_moss.pkl")
```

In [ ]:
preds = self.annotate_clusters(model="COVID19_HumanChallenge_Blood.pkl")
# self._columns["col_cell_type"] = "predicted_labels"  # use by default from now on
self.obs_rna

### Find Cluster Markers (DEGs)

To sort by absolute values on "scores" for cluster 6, showing the top 3 gene markers:

`clus = "6"`

`marks.loc[clus].loc[marks.loc[clus].scores.abs().sort_values(ascending=False).iloc[:3].index]`

To show the top 3 gene markers for all clusters:

`marks.groupby(marks.index.names[0]).apply(lambda x: x.scores.abs().sort_values(ascending=False).iloc[:3].index])`

In [ ]:
degs = self.find_markers(n_genes=10, method="wilcoxon", key_reference="rest")
degs[0]  # degs[1] for figures

## Inspect & Prepare

## Explore Data Descriptives

In [ ]:
_ = self.describe()  # simple
# _ = self.describe(group_by=self._columns["col_target_genes"], plot=True)

## Set Up Arguments for Later


This code looks more complicated than it actually would actually be for an end user because it was made to be generalizable across several datasets with particular column names, sizes that make it necessary to subset them in order to run the vignettes in a reasonable period of time, etc.

Basically, you won't need this code as an end user; this is just to choose random subsets of genes and perturbations, etc. that are available in a given example dataset.

In real use cases, you will know what genes and conditions are of interest, and you can manually specify them by simply stating them in the appropriate arguments (such as `target_gene_idents`) or (in many cases) by not specifying the argument (resulting in the code using all available genes, etc.).

In [ ]:
genes = np.array(self.rna.var.reset_index()[self._columns["col_gene_symbols"]])
if file == "CRISPRi_scr":
    genes_subset = ["CDKN2A", "CDKN1A", "PLAUR", "TP53", "IL6ST", "IL1A", 
                    "CXCL8", "CCL2", "CEBPB", "NFKB1", "IGFBP7", "TGFB1", 
                    "BCL2", "PTGER4", "ERCC1", "ERCC4", "ATG16L1", "NOD2", "LACC1"]
else:
    genes_subset = list(pd.Series(genes).sample(10))
target_gene_idents = list(self.obs[self._columns["col_target_genes"]].sample(
    10))  # 10 random guide targets

# Initial Plots

## Basic Usage

You can create simple plots easily without having to remember a bunch of arguments to specify! 

The most useful is the `genes` argument, which allows you to subset the number of features plotted (useful for spead and layout/interpretability of plots).

In [ ]:
figs = self.plot(genes=genes_subset)

## Advanced Usage

Use the `layers` argument to plot more layers (in this instance, all of them, including the scaled data) for certain plot types.
 
Use the `cell_types_circle` argument to create a UMAP with certain cell types circled in red.

Use the `genes_highlight` argument to highlight in gold the names of specified genes on the gene expression dot plot.

Use the `kws_clustering` argument to specify a dictionary keywords to pass to certain UMAP-based plots. For instance, specify `kws_clustering=dict(col_cell_type="leiden")` to use leiden clusters instead of whatever is stored in `self._columns["col_cell_type"]`.

Use the `kws_gex_violin` argument to pass additional arguments to the violin plots of gene expression.

In [ ]:
cct = "predicted_labels" if "predicted_labels" in (
    self.adata[assay] if self._assay else self.adata).obs else self._columns[
        "col_cell_type"]  # cell type column to use
clus = list((self.adata[self._assay] if self._assay else self.adata).obs[
    cct].sample(2))  # clusters to circle
figs = self.plot(genes=genes_subset, 
                 col_cell_type=cct,
                 layers="all",  # also plot "scaled" layer, for instance
                 cell_types_circle=clus,  # list cell types to circle on UMAP
                 genes_highlight=list(np.array(genes_subset)[1:3]), 
                 kws_gex_violin=dict(scale="area", height=10),
                 kws_clustering=dict(col_cell_type=cct))

# Perturbation Analyses

## Guide RNAs

### Guide RNA Counts/Percentage

In [ ]:
tgis = list(pd.Series(target_gene_idents).sample(3)) if len(
    target_gene_idents) > 3 else target_gene_idents  # smaller subset for speed
self.get_guide_rna_counts(target_gene_idents=tgis)
self.get_guide_rna_counts(target_gene_idents=tgis, 
                          group_by=True)  # by cell type
self.get_guide_rna_counts(target_gene_idents=tgis, group_by=[
    self._columns["col_cell_type"], self._columns["col_target_genes"]], 
                          margin_titles=True)
self.get_guide_rna_counts(target_gene_idents=tgis, group_by=[
    self._columns["col_target_genes"]], margin_titles=True)

### Targeting Efficiency

In [53]:
col_num_umis="num_umis"

In [51]:
feature_split="|"

In [63]:
tg_info

[autoreload of crispr.crispr_class failed: Traceback (most recent call last):
  File "/home/asline01/anaconda3/envs/py-bio/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/home/asline01/anaconda3/envs/py-bio/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 475, in superreload
    module = reload(module)
  File "/home/asline01/anaconda3/envs/py-bio/lib/python3.10/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 619, in _exec
  File "<frozen importlib._bootstrap_external>", line 879, in exec_module
  File "<frozen importlib._bootstrap_external>", line 1017, in get_code
  File "<frozen importlib._bootstrap_external>", line 947, in source_to_code
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/home/asline01/projects/crispr/crispr/crispr_class.py", line 365
    elif col_p

,feature_call,feature_call_flat_ix,feature_call_list_all,num_umis_list_all,num_umis_total,feature_call_list_filtered,num_umis_list_filtered,feature_call_filtered,num_umis_filtered,feature_split,num_umis
AAACCCAAGATTTGCC-1,SNX17,SNX17-1_2,"[SNX17, SNX17]","[200.0, 33.0]",233.0,[SNX17],[233.0],SNX17,233.0,|,233.0
AAACCCAAGCGATTCT-1,HLA,SP110-1|HLA-B-1|SP140L-2,"[SP110, HLA, SP140L]","[12.0, 22.0, 7.0]",41.0,[HLA],[22.0],HLA,22.0,|,22.0
AAACCCAAGTCTGCAT-1,RSBN1,RSBN1-1_2|PRDX6-1_2|STK11-1|NR4A1-2,"[RSBN1, PRDX6, STK11, RSBN1, PRDX6, NR4A1]","[194.0, 381.0, 51.0, 513.0, 119.0, 47.0]",1305.0,[RSBN1],[707.0],RSBN1,707.0,|,707.0
AAACCCACAGGATCTT-1,NR4A1,TAGAP-1|CRYZL1-1_2|MYD88-1|NR4A1-1_2|SPI1-1|IF...,"[TAGAP, CRYZL1, MYD88, NR4A1, SPI1, IFNAR2, IL...","[378.0, 47.0, 29.0, 503.0, 9.0, 41.0, 84.0, 26...",1201.0,[NR4A1],[587.0],NR4A1,587.0,|,587.0
AAACCCAGTCTCAAGT-1,SPI1,SPI1-2,[SPI1],[11.0],11.0,[SPI1],[11.0],SPI1,11.0,|,11.0
...,...,...,...,...,...,...,...,...,...,...,...
TTTGTTGAGGTGCAGT-1,FASLG,FASLG-1_2,"[FASLG, FASLG]","[58.0, 41.0]",99.0,[FASLG],[99.0],FASLG,99.0,|,99.0
TTTGTTGAGTACTCGT-1,POLR2E,POLR2E-1_2|IFNGR2-1_2,"[POLR2E, IFNGR2, POLR2E, IFNGR2]","[681.0, 216.0, 146.0, 143.0]",1186.0,[POLR2E],[827.0],POLR2E,827.0,|,827.0
TTTGTTGGTTGCCGAC-1,DUSP1,DUSP1-1_2|PPARA-1|TAGAP-2,"[DUSP1, PPARA, TAGAP, DUSP1]","[48.0, 19.0, 4.0, 46.0]",117.0,[DUSP1],[94.0],DUSP1,94.0,|,94.0
TTTGTTGTCACTGTTT-1,GPX4,ATG16L1-1|GPX4-1,"[ATG16L1, GPX4]","[73.0, 125.0]",198.0,[GPX4],[125.0],GPX4,125.0,|,125.0


## Augur: Cell Type-Level Perturbation Response

Which cell types are most affected by perturbations? Quantify perturbation responses by cell type with Augur, which uses supervised machine learning classification of experimental condition labels (e.g., treated versus untreated). The more separable the condition among cells of a given type, the higher the perturbation effect score.

<u> __Features__ </u>  

- Quantify and visualize degree of perturbation response by cell type
- Identify the most important features (genes).

<u> __Input__ </u>  

<u> __Output__ </u>  

mean_augur_score

<u> __Notes__ </u>  

- Sub-sample sizes equal across conditions; does not account for perturbation-induced compositional shifts (cell type abundance)
- Scores are for cell types (aggregated across cells, not individual cells)
- Two modes
    - If select_variance_feature=True, run the original Augur implementation, which removes genes that don't vary much across cell type.
    - If False, use features selected by `scanpy.pp.highly_variable_genes()`, which is faster and sensitively recovers effects; however, the feature selection may yield inflated Augur scores because this reduced feature set is used in training, resulting in it taking advantage of the pre-existing power of this feature selection to separate cell types.
    - If False, you also have to be sure that "highly_variable_features" is a variable in your data. This can be complicated if you have a separate layer for perturbation data.

In [32]:
augur_data.obs = augur_data.obs.rename({"y_": })

[autoreload of crispr.crispr_class failed: Traceback (most recent call last):
  File "/home/asline01/anaconda3/envs/py-bio/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/home/asline01/anaconda3/envs/py-bio/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 475, in superreload
    module = reload(module)
  File "/home/asline01/anaconda3/envs/py-bio/lib/python3.10/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 619, in _exec
  File "<frozen importlib._bootstrap_external>", line 879, in exec_module
  File "<frozen importlib._bootstrap_external>", line 1017, in get_code
  File "<frozen importlib._bootstrap_external>", line 947, in source_to_code
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/home/asline01/projects/crispr/crispr/crispr_class.py", line 352
    elif col_p

AnnData object with n_obs × n_vars = 6586 × 2368
    obs: 'num_features', 'feature_call_original', 'num_umis_original', 'feature_call', 'feature_call_flat_ix', 'feature_call_list_all', 'num_umis_list_all', 'num_umis_total', 'feature_call_list_filtered', 'num_umis_list_filtered', 'feature_call_filtered', 'num_umis_filtered', 'feature_split', 'num_umis', 'target_gene_name', 'perturbation', 'n_genes', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'leiden', 'label', 'cell_type', 'batch', 'y_', 'augur_score'
    var: 'gene_ids', 'feature_types', 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
    uns: 'summary_metrics', 'augurpy_results', 'leiden_colors', 'neighbors', 'umap'
    obsm: 'X_pca', 'X_umap'
    layers: 'counts', 'raw

In [ ]:
cct = "majority_voting"  # celltypist
# cct = "predicted_labels"  # celltypist
# cct = None  # to use self._columns["col_cell_type"]

augur_adata, augur_results, figs_augur = self.run_augur(
    col_perturbed=self._columns["col_perturbed"], 
    col_cell_type=cct,
    key_treatment=self._keys["key_treatment"], 
    classifier="random_forest_classifier", n_threads=True,
    augur_mode="default", select_variance_features=True, 
    n_folds=3,
    kws_umap=kws_umap, subsample_size=5, 
    kws_augur_predict=dict(span=0.7))

## Mixscape: Cell-Level Perturbation Response

**Which cell types are most affected by perturbations?** Quantify perturbation responses by cell type with Augur, which uses supervised machine learning classification of condition labels (e.g., treated versus untreated) and looking at how easy it is to tell perturbed versus non-perturbed (more separable = higher effect).

**Are there perturbation-specific clusters?** Mixscape uses linear discriminant analysis (LDA) to cluster cells that resemble each other in terms of gene expression and perturbation condition. _(LDA reduces dimensionality and attempts to maximize the separability of classes. Unperturbed cells are removed from analysis.)_

<u> __Features__ </u>  

- Remove confounds (e.g., cell cycle, batch effects)
- Classify cells as affected or unaffected (i.e., "escapees") by the perturbation
- Quantify and visualize degree of perturbation response

<u> __Input__ </u> 

See documentation, but the key arguments are listed here.

* **col_cell_type**: If you want to run using a different cell classification column, (e.g., CellTypist annotations that weren't used for the original `self._columns["col_cell_type"]``), you can specify a different column by passing `col_cell_type=<column name>` if you'd like.
* **target_gene_idents**: A list of gene symbols to focus on in plots/analyses. True to include all.

<u> __Output__ </u>  
- Mixscape figures

<u> __Notes__ </u>  

In [ ]:
cct = "predicted_labels" if "predicted_labels" in self.obs.columns else None
figs_mixscape = self.run_mixscape(
    col_split_by=col_split_by,  # to split by, e.g, replicates if not None
    col_cell_type=cct,  # use cell type column specified above 
    target_gene_idents=True)  # to use all; otherwise, specify gene list

## Distance Metrics

See `self.figures[run_label]["distances"]` and  `self.results[run_label]["distances"]`
for results.

In [ ]:
cct = "predicted_labels" if "predicted_labels" in self.adata.obs.columns else None
fig_dist_w = self.compute_distance(distance_type="edistance", method="X_pca",
                                   col_cell_type=cct,
                                   kws_plot=dict(figsize=(15, 15), robust=True))

fig_dist_e = self.compute_distance(distance_type="mmd", method="X_pca",
                                   col_cell_type=cct,
                                   kws_plot=dict(figsize=(15, 15), robust=True))

## Composition Analysis

In [ ]:
try:
    oca = self.run_composition_analysis(
        "0", est_fdr=0.05, generate_sample_level=False)
except Exception as err:
    print(f"{err}\n\n\nComposition analysis failed!")

## Dialogue: Multi-Cellular Programs

In [ ]:
fig_mcp = self.run_dialogue(n_programs=4, cmap="coolwarm")

# Random Workspace (IGNORE)

In [ ]:
import scanpy as sc
import matplotlib.pyplot as plt

sc.pl.umap(self.adata, color="CDKN1A", palette="coolwarm", legend_loc="on_data", show=False)
a_x = plt.gca()
for cluster in self.adata.obs[col_cell_type].unique():
    cmu = self.adata[self.adata.obs[col_cell_type] == cluster].obsm[
        "X_umap"].mean(axis=0)
    a_x.annotate(cluster, cmu, fontsize=8, color="black", 
                 weight="bold", ha="center")
plt.show()


for ax in figs["clustering_gene_expression"].axes:
    for cluster in self.adata.obs[col_cell_type].unique():
        cmu = self.adata[self.adata.obs[col_cell_type] == cluster].obsm[
            "X_umap"].mean(axis=0)
        ax.annotate(cluster, cmu, fontsize=8, color="black", 
                    weight="bold", ha="center")

In [ ]:
import crispr as cr
import numpy as np

file = "/home/asline01/projects/crispr/examples/data/adamson_2016_upr_perturb_seq.h5ad"
kwargs_init = {
    "assay": None, "assay_protein": None, "col_cell_type": "celltype",
    "col_gene_symbols": "gene_symbol", "col_sample_id": None, 
    "col_batch": None, "col_perturbed": "perturbation", 
    "col_guide_rna": "perturbation", "col_num_umis": "UMI count", 
    "kws_process_guide_rna": dict(
        feature_split=None, guide_split="_", key_control_patterns=[np.nan]), 
    "col_target_genes": "perturbation", "key_control": "Control", 
    "key_treatment": "KO", "remove_multi_transfected": True}
for q in [kwargs_init, kwargs_init["kws_process_guide_rna"]]:
    cr.tl.print_pretty_dictionary(q)
adata = cr.pp.create_object(file, **kwargs_init)

ann = adata.copy()

assay=np.nan
assay_protein=np.nan
col_cell_type='celltype'
col_gene_symbols='gene_symbol'
col_sample_id=np.nan
col_batch=np.nan
col_perturbed='perturbation'
col_guide_rna='feature_call'
col_num_umis='UMI count'
kws_process_guide_rna=dict(feature_split = np.nan, guide_split = '_', key_control_patterns = [np.nan])
col_target_genes='perturbation'
key_control='Control'
key_treatment='KO'
remove_multi_transfected=True
feature_split=np.nan
guide_split='_'
key_control_patterns=[np.nan]
max_percent_umis_control_drop=75
min_percent_umis=40

feature_split = "|"
if ann.obs[col_guide_rna].apply(lambda x: feature_split in x).any():
    raise ValueError(
        f"""For single-guide designs, the character {feature_split}
        cannot be found in any of the guide names ({col_guide_rna})""")
tg_info = cr.pp.find_guide_info(
    ann.copy(), col_guide_rna, col_num_umis=col_num_umis, 
    feature_split=feature_split, guide_split=guide_split, 
    key_control_patterns=key_control_patterns, 
    key_control=key_control)



out = cr.pp.filter_by_guide_counts(
    adata, kwargs_init["col_guide_rna"], kwargs_init["col_num_umis"],
    max_percent_umis_control_drop=75,
    min_percent_umis=40, **kwargs_init["kws_process_guide_rna"], 
    key_control="Control")